# Lab05 - SMT solving in Python with Z3

Satisfiability modulo theory (SMT)

SMT solvers such as Z3 generalise SAT solvers from the boolean domain to other universes, such as reals or integers. Quantifiers are also supported. Available theories in Z3:

* LIA: Linear integer arithmetic (Presburger's logic) (ℤ,<,+,0,1).
* NIA: Non-linear integer arithmetic (ℤ,<,+,⋅,0,1).
* LRA: Linear real arithmetic (ℝ,<,+,(⋅*𝜆*)*𝜆*∈ℝ,0,1).
* NRA: Non-linear real arithmetic (Tarski's algebra) (ℝ,<,+,⋅,0,1).
* Arrays.
* BV: Bit-vectors.
* AD: Algebraic datatypes (eg. lists, trees, graphs).
* Sequences and strings.
* EUF: Equality and uninterpreted functions.

# Integers (NIA)

**Example \[Diophantine equations\].** Diophantine equations are undecidable in general, but Z3 is able to solve some particular ones. Is Z3 is able to check whether the following are satisfiable over the integers:

$$
\begin{align} (1)\quad &x^2+y^2=z^2, \quad x, y > 1; \\ (2)\quad &x^3+y^3=z^3, \quad x, y > 1; \\ (3)\quad &x^2+y^2=4z+3; \\ (4)\quad &x^2+y^2=4z+2. \end{align}
$$


In [1]:
from z3 import *
set_param("timeout", 5_000)  # in miliseconds

x, y, z = Ints('x y z')
print("(1)")
solve(x > 0, y > 0, z > 0, x*x + y*y == z*z)

print("(2)") # not satisfiable; "failed to solve" means Z3 gave up
solve(x > 0, y > 0, z > 0, x*x*x + y*y*y == z*z*z)

print("(3)") # not satisfiable; "failed to solve" means Z3 gave up
solve(x*x + y*y == 4*z + 3)

# even this is too hard...
solve(ForAll(x,Implies(Exists(y, x == 2 * y), Exists(z, x * x == 4 * z))))
solve(Not(Implies(Exists(y, x == 2 * y), Exists(z, x * x == 4 * z))))

solve(Not(Implies(x == 2 * y, x * x == 4 * y * y)))
solve(Not(Implies(x == 2 * y, Exists(z, x * x == 4 * z))))

solve(Not(Implies(x == 2 * y + 1, Exists(z, x * x == 4 * z + 1))))
solve(Not(Exists(y, Or(x == 2*y, x == 2*y+1))))

print("(4)") # easy :)
solve(x*x + y*y == 4*z + 2)

(1)
[z = 13, x = 5, y = 12]
(2)
failed to solve
(3)
failed to solve
failed to solve
failed to solve
no solution
no solution
no solution
no solution
(4)
[x = -3, z = 2, y = -1]


# Reals (LRA & NRA) 

Z3 can solve some liniear and non-linear (in)equalities over reals.

In [2]:
from z3 import *

# Linear (in)equalities
x, y, z = Reals('x y z')
phi = lambda x, y: And(x < 0, 3*x + 5*y <= 1, 2*x + 3*y > 0)

print("Is there any solution?")
solve(phi(x, y))

print("Closed theorem?")
psi = ForAll([x,y], Implies(x < y, Exists(z, And(x < z, z < y))))
solve(psi)
print("Let us negate it")
npsi = Exists([x,y],And(x < y, ForAll(z, Or(z <= x, y <= z))))
solve(npsi)
print("We can omit Exist - it is a solver after all")
npsi = And(x < y, ForAll(z, Or(z <= x, y <= z)))
solve(npsi)

# Non-linear (in)equalities
phi = lambda x, y: And(x > 1, y > 1, x*x + y*y  <= 7, x*x - y  <= 2)
print("Is there any solution?")
solve(phi(x, y))

Is there any solution?
[y = 1/4, x = -1/4]
Closed theorem?
[]
Let us negate it
no solution
We can omit Exist - it is a solver after all
no solution
Is there any solution?
[y = 2, x = 3/2]


# Equality and Uninterpreted Functions (EUF)

One can also try to solve problems from abstract first order logic.

In [3]:
from z3 import *
set_param("timeout", 5_000) ## in miliseconds

# we can declare new (non-empty) sorts
X = DeclareSort('X')

# we can declare variables of sort X
x, y, z = Consts('x y z', X)

# unsatisfiable since domains such as X are always non-empty
print("Can the model be empty?")
solve(ForAll(x, x != x))

# we can also declare functions
f = Function('f', X, X)

# we can express constraints on f
print("\nIs there a non-constant function?")
constant = Exists(y, ForAll(x, f(x) == y))
solve(Not(constant))

print("\nIs there an identity function?")
identity = ForAll(x, f(x) == x)
solve(identity)

print("\nIs there a constant identity ?")
solve(identity, constant)

print("\nIs there a constant identity with a model with at least two elements?")
solve(identity, constant, x != y)

print("\nIs there an identity with a model with at least two elements?")
solve(identity, x != y)


print("\nIf fg = gf, can ffg != gff?")
g = Function('g', X, X)
commute = ForAll(x, f(g(x)) == g(f(x)))
solve(commute, f(f(g(x))) != g(f(f(x))))
print("Can ffg != gfg?")
solve(commute, f(f(g(x))) != g(f(g(x))))


Can the model be empty?
no solution

Is there a non-constant function?
[f = [X!val!3 -> X!val!4,
      X!val!5 -> X!val!6,
      else -> X!val!2]]

Is there an identity function?
[f = [else -> Var(0)]]

Is there a constant identity ?
[f = [else -> Var(0)]]

Is there a constant identity with a model with at least two elements?
no solution

Is there an identity with a model with at least two elements?
[y = X!val!1, x = X!val!0, f = [else -> Var(0)]]

If fg = gf, can ffg != gff?
no solution
Can ffg != gfg?
failed to solve


# Exercise (Łoś)

Let $X$ be a set. The so-called *Łoś principle* involves two binary relations $P, Q \subseteq X \times X$ and states the following: If $P, Q$ are transitive, $P$ is symmetric, and $P \cup Q = X \times X$, then either $P = X \times X$ or $Q = X \times X$. 

Does this principle hold? Are all premises necessary? Model this problem in Z3 and use it to verify your intuition.

In [4]:
from z3 import *

X = DeclareSort('X') 
B = BoolSort()

# binary predicates P, Q: X×X → B
P = Function('P', X, X, B)   
Q = Function('Q', X, X, B)

x, y = Consts("x y", X)

# some examples
solve(ForAll(x,Q(x,x)))
solve(ForAll(x,Not(Q(x,x))))
solve(ForAll(x,Exists(y, And(Not(x==y), Q(x,y)))))
solve(ForAll([x,y], Q(x,y) == Not(Q(y,x))))

# since relation definitions are hard to follow we can as well not look at them...
def test(phi):
  S = Solver()
  S.add(phi)
  ans = S.check()
  return False if ans == unsat else True if ans == sat else None
  #print("No solution" if ans == unsat else "[ ..solution.. ]" if ans == sat else "Failed to solve")

print("solvable? ", test(ForAll(x,Exists(y, And(Not(x==y), Q(x,y))))))

transitiveP = ForAll([x, y, z], Implies(And(P(x, y), P(y, z)), P(x, z)))
transitiveQ = ForAll([x, y, z], Implies(And(Q(x, y), Q(y, z)), Q(x, z)))
symmetricP = ForAll([x, y], Implies(P(x, y), P(y, x)))
covering = ForAll([x, y], Or(P(x, y), Q(x, y)))
fullRelationPQ = Or(ForAll(x, ForAll(y, P(x, y))), ForAll(x, ForAll(y, Q(x, y))))

theorem = Implies(And(transitiveP, transitiveQ, symmetricP, covering), fullRelationPQ) # All premises necessary

s = Solver()
s.add(Not(theorem))
res = s.check()

if res == unsat:
    print("The Łoś principle holds")
elif res == sat:
    print("The Łoś principle does not hold")
else:
    print("Solver failed to determine")

[Q = [else -> True]]
[Q = [else -> False]]
[Q = [else ->
      Or(And(Not(Var(0) == X!val!3),
             Not(Var(0) == X!val!2),
             Not(Var(0) == X!val!1),
             Not(Var(0) == X!val!5),
             Not(Var(0) == X!val!0),
             Var(1) == X!val!5),
         And(Var(0) == X!val!2,
             Not(Var(0) == X!val!1),
             Not(Var(0) == X!val!5),
             Not(Var(0) == X!val!0),
             Var(1) == X!val!3),
         And(Var(0) == X!val!0, Var(1) == X!val!1),
         And(Var(0) == X!val!3,
             Not(Var(0) == X!val!2),
             Not(Var(0) == X!val!1),
             Not(Var(0) == X!val!5),
             Not(Var(0) == X!val!0),
             Var(1) == X!val!4),
         And(Var(0) == X!val!5,
             Not(Var(0) == X!val!0),
             Var(1) == X!val!6),
         And(Var(0) == X!val!1,
             Not(Var(0) == X!val!5),
             Not(Var(0) == X!val!0),
             Var(1) == X!val!2))]]
no solution
solvable?  True
The Łoś princ

# **Exercise (Function 91)**

The following is a convoluted recursive definition of an integer function, called *function 91* (a.k.a. *McCarthy's function*): 

$$
\begin{align} \textrm{mc}(n) = \left\{\begin{array}{ll} n - 10 & \textrm { if } n > 100, \\ \textrm{mc}(\textrm{mc}(n + 11)) & \textrm { otherwise.} \end{array}\right. \end{align}
$$
It turns out that the function above is total, and that it can equivalently be defined in the following non-recursive fashion: 

$$
\begin{align} f(n) = \left\{\begin{array}{ll} k & \textrm { if } n \leq n_0, \\ n + b & \textrm { otherwise.} \end{array}\right. \end{align} 
$$
Use Z3 to *synthesise* the parameters $k, b, n_0 \in \mathbb Z$ s.t. $f = mc$.

In [5]:
from z3 import *
Z = IntSort()
mc = Function('mc', Z, Z)
n, k, b, n0 = Consts('n k b n0', Z)

mc_def = [
    ForAll(n, Implies(n > 100, mc(n) == n - 10)),
    ForAll(n, Implies(n <= 100, mc(n) == mc(mc(n + 11)))),
    ForAll(n, Implies(n > n0, mc(n) == n + b)),
    ForAll(n, Implies(n <= n0, mc(n) == k))
]

s = Solver()
s.add(mc_def)
res = s.check()

if res == sat:
    m = s.model()
    print("Soultion found:")
    print("k =", m[k])
    print("b =", m[b])
    print("n0 =", m[n0])
else:
    print("No solution")

Soultion found:
k = 91
b = -10
n0 = 101


# Exercise \[Sudoku\]

Solve a Sudoku puzzle using Z3. You have to fill the dots in a 9x9 grid with digits from '1' to '9' in such a way that:

* in each row, every number appears exactly once,
* in each column, every number appears exactly once,
* our grid consists of nine 3x3 squares; in each of them, every number has to appear exactly once.

Pro snippet: `Distinct(x,y,z,...)` imposes that the given values are pairwise distinct.

In [6]:

puzzle = [
"..53.....",
"8......2.",
".7..1.5..",
"4....53..",
".1..7...6",
"..32...8.",
".6.5....9",
"..4....3.",
".....97.."]

dim = 3
dim2 = dim**2

# varibles: X[i][j]
X = [[Int(f'x{i}{j}') for i in range(dim2)] for j in range(dim2)]

def model(phi):
    # create a SAT instance
    s = Solver()
    s.add(phi)    
    # return a satisfying assignment
    return s.model() if s.check() == sat else {}

def display(sol):
    if not sol:
        print("No solution...")
        return
    for i in range(dim2):
        for j in range(dim2):
            print("|", end='') if (j % dim == 0 and j > 0) else print("", end='')
            print(sol[X[i][j]], end='')
        print("")

        if i % dim == dim-1 and i > 0 and i < dim2-1:
            for j in range(dim2 + dim - 1):
                print("+", end='') if (j % 4 == dim and j > 0) else print("-", end='')
            print("")

domain = And([And(1 <= X[i][j], X[i][j] <= dim2) 
              for j in range(dim2) for i in range(dim2)])

given = And([X[i][j] == int(puzzle[i][j]) if puzzle[i][j] != '.' else True
              for j in range(dim2) for i in range(dim2)])

row_constraint = And([Distinct(X[i]) for i in range(dim2)])

col_constraint = And([Distinct([X[i][j] for i in range(dim2)]) for j in range(dim2)])

square_constraint = And([Distinct([X[i + di][j + dj] for di in range(dim) for dj in range(dim)])
                       for i in range(0, dim2, dim) for j in range(0, dim2, dim)])

phi = And(domain, given, row_constraint, col_constraint, square_constraint)
sol = model(phi)

display(sol)

145|327|698
839|654|127
672|918|543
---+---+---
496|185|372
218|473|956
753|296|481
---+---+---
367|542|819
984|761|235
521|839|764
